<a href="https://colab.research.google.com/github/PatronesGrupo4/Grupo4/blob/main/Challenges/Challenge_2__avance_Sergio.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

$$\Large \textit{C0708 | Reconocimiento de Patrones}$$

$$\large \textbf{Challenge 2 | Métodos de aprendizaje supervisado: Regresión lineal, polinomial, lineal múltiple, KNN}$$

# Integrantes Grupo N°4

- Alonso Santiago Morán Espinoza
- Sergio Enrique Moreno Elescano

# Análisis exploratorio

## Importación de librerías

En primer lugar se importarán las librerías a utilizar y se cargará el archivo con el cual se trabajará

In [136]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import sklearn
import seaborn as sns

df_infecciones = pd.read_csv('DIAGNOSTICOS_POR_GRUPOS_DE_EDAD.csv', delimiter=';')
df_infecciones.head()

,PERIODO_DIAGNOSTICO,ESTABLECIMIENTO,DEPARTAMENTO,PROVINCIA,DISTRITO,UBIGEO,FECHA_CORTE,SEXO,GRUPOS_POR_EDAD,DIAGNOSTICOS_CIE,CATEGORIA_DIAGNOSTICO,TOTAL_PACIENTES
0,202301,HOSPITAL NACIONAL ARZOBISPO LOAYZA,LIMA,LIMA,LIMA,150101,20230615,FEMENINO,DE 25 A 29 AÑOS,INFECCIÓN POR SALMONELLA,OTRAS INFECCIONES DEBIDAS A SALMONELLA,1
1,202301,HOSPITAL NACIONAL ARZOBISPO LOAYZA,LIMA,LIMA,LIMA,150101,20230615,FEMENINO,MENORES DE 1 AÑO,OTRAS INFECCIONES INTESTINALES BACTERIANAS ESP...,OTRAS INFECCIONES INTESTINALES BACTERIANAS,1
2,202301,HOSPITAL NACIONAL ARZOBISPO LOAYZA,LIMA,LIMA,LIMA,150101,20230615,MASCULINO,DE 60 A 64 AÑOS,ABSCESO AMEBIANO DEL PULMÓN,AMEBIASIS,2
3,202301,HOSPITAL NACIONAL ARZOBISPO LOAYZA,LIMA,LIMA,LIMA,150101,20230615,MASCULINO,DE 65 AÑOS A MÁS,"AMEBIASIS, NO ESPECIFICADA",AMEBIASIS,1
4,202301,HOSPITAL NACIONAL ARZOBISPO LOAYZA,LIMA,LIMA,LIMA,150101,20230615,FEMENINO,DE 5 A 9 AÑOS,GIARDIASIS [LAMBLIASIS],OTRAS ENFERMEDADES INTESTINALES DEBIDAS A PROT...,2


## Descripción general del dataset

A continuación, se muestra la información más relevante del dataset

In [137]:
print(df_infecciones.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6049 entries, 0 to 6048
Data columns (total 12 columns):
 #   Column                 Non-Null Count  Dtype 
---  ------                 --------------  ----- 
 0   PERIODO_DIAGNOSTICO    6049 non-null   int64 
 1   ESTABLECIMIENTO        6049 non-null   object
 2   DEPARTAMENTO           6049 non-null   object
 3   PROVINCIA              6049 non-null   object
 4   DISTRITO               6049 non-null   object
 5   UBIGEO                 6049 non-null   int64 
 6   FECHA_CORTE            6049 non-null   int64 
 7   SEXO                   6049 non-null   object
 8   GRUPOS_POR_EDAD        6049 non-null   object
 9   DIAGNOSTICOS_CIE       6049 non-null   object
 10  CATEGORIA_DIAGNOSTICO  6049 non-null   object
 11  TOTAL_PACIENTES        6049 non-null   int64 
dtypes: int64(4), object(8)
memory usage: 567.2+ KB
None


Se pueden observar que se cuentan con 12 columnas, de estas se seleccionarán los features más relevantes así como también el target a predecir.
Las columnas corresponden a las siguientes características:

- Periodo Diagnóstico: Es el período del año en el cual se realizó el diagnóstico.
- Establecimienot: Corresponde al establecimiento de salud en donde se realizó el diagnóstico.
- Departamento: Es el departamento del país en el cual se obutvo el diagnóstico.
- Provincia: Es la provincia del departamento donde se obtuvo el diagnóstico.
- Ubigeo: Corresponde a la localidad del diagnóstico.
- Fecha Corte: Indica el corte en la fecha de la recopilación de los datos.
- Sexo
- Grupos por edad: Es el rango de edad de los pacientes que fueron diagnosticados.
- Diagnósticos CIE: Es el diagnóstico según la normativa CIE 10.
- Categoría de diagnóstico.
- Total pacientes: Es la cantidad de pacientes diagnósticados.

A continuación, se muestra la descripción estadística de los datos.

In [138]:
df_infecciones.describe()

,PERIODO_DIAGNOSTICO,UBIGEO,FECHA_CORTE,TOTAL_PACIENTES
count,6049.0,6049.0,6049.0,6049.000000
mean,202301.0,150101.0,20230615.0,2.610845
std,0.0,0.0,0.0,6.264612
min,202301.0,150101.0,20230615.0,1.000000
25%,202301.0,150101.0,20230615.0,1.000000
50%,202301.0,150101.0,20230615.0,1.000000
75%,202301.0,150101.0,20230615.0,2.000000
max,202301.0,150101.0,20230615.0,247.000000


# Encoding

Utilizaremos un Ordinal Encoder para procesar los datos cualitativos del dataset. Primero, es conveniente definir cual será el target del algoritmo. 
Analizando los datos del dataframe, consideramos conveniente seleccionar como target a predecir a la categoría "CATEGORIA_DIAGNÓSTICO". Además, es evidente que no todas las características del dataset resultan útiles. Las características a utilizar serán las siguientes: "GRUPOS_POR_EDAD", "DIAGNÓSTICO_CIE".

## Encoding de la característica "GRUPOS_POR_EDAD"

In [139]:
from sklearn import preprocessing
from sklearn.preprocessing import OrdinalEncoder
encoder = OrdinalEncoder(categories=[['MENORES DE 1 AÑO', 'DE 1 A 4 AÑOS', 'DE 5 A 9 AÑOS', 'DE 10 A 14 AÑOS', 'DE 15 A 19 AÑOS', 'DE 20 A 24 AÑOS', 'DE 25 A 29 AÑOS', 'DE 30 A 34 AÑOS', 'DE 35 A 39 AÑOS', 'DE 40 A 44 AÑOS', 'DE 45 A 49 AÑOS', 'DE 50 A 54 AÑOS', 'DE 55 A 59 AÑOS', 'DE 60 A 64 AÑOS', 'DE 65 AÑOS A MÁS']])
df_infecciones_clean = df_infecciones
df_infecciones_clean['GRUPOS_EDAD_ENCODED'] = encoder.fit_transform(df_infecciones_clean[['GRUPOS_POR_EDAD']])
df_infecciones_clean.head()

,PERIODO_DIAGNOSTICO,ESTABLECIMIENTO,DEPARTAMENTO,PROVINCIA,DISTRITO,UBIGEO,FECHA_CORTE,SEXO,GRUPOS_POR_EDAD,DIAGNOSTICOS_CIE,CATEGORIA_DIAGNOSTICO,TOTAL_PACIENTES,GRUPOS_EDAD_ENCODED
0,202301,HOSPITAL NACIONAL ARZOBISPO LOAYZA,LIMA,LIMA,LIMA,150101,20230615,FEMENINO,DE 25 A 29 AÑOS,INFECCIÓN POR SALMONELLA,OTRAS INFECCIONES DEBIDAS A SALMONELLA,1,6.0
1,202301,HOSPITAL NACIONAL ARZOBISPO LOAYZA,LIMA,LIMA,LIMA,150101,20230615,FEMENINO,MENORES DE 1 AÑO,OTRAS INFECCIONES INTESTINALES BACTERIANAS ESP...,OTRAS INFECCIONES INTESTINALES BACTERIANAS,1,0.0
2,202301,HOSPITAL NACIONAL ARZOBISPO LOAYZA,LIMA,LIMA,LIMA,150101,20230615,MASCULINO,DE 60 A 64 AÑOS,ABSCESO AMEBIANO DEL PULMÓN,AMEBIASIS,2,13.0
3,202301,HOSPITAL NACIONAL ARZOBISPO LOAYZA,LIMA,LIMA,LIMA,150101,20230615,MASCULINO,DE 65 AÑOS A MÁS,"AMEBIASIS, NO ESPECIFICADA",AMEBIASIS,1,14.0
4,202301,HOSPITAL NACIONAL ARZOBISPO LOAYZA,LIMA,LIMA,LIMA,150101,20230615,FEMENINO,DE 5 A 9 AÑOS,GIARDIASIS [LAMBLIASIS],OTRAS ENFERMEDADES INTESTINALES DEBIDAS A PROT...,2,2.0


## Encoding de la característica "CATEGORIA_DIAGNOSTICO"

In [140]:
categorias_diag = []
for elemento in df_infecciones['DIAGNOSTICOS_CIE']:
    if elemento not in categorias_diag:
        categorias_diag.append(elemento)

encoder = OrdinalEncoder(categories=[categorias_diag])
df_infecciones_clean = df_infecciones
df_infecciones_clean['GRUPOS_DIAGNOSTICOS_CIE_ENCODED'] = encoder.fit_transform(df_infecciones_clean[['DIAGNOSTICOS_CIE']])
df_infecciones_clean.head()

,PERIODO_DIAGNOSTICO,ESTABLECIMIENTO,DEPARTAMENTO,PROVINCIA,DISTRITO,UBIGEO,FECHA_CORTE,SEXO,GRUPOS_POR_EDAD,DIAGNOSTICOS_CIE,CATEGORIA_DIAGNOSTICO,TOTAL_PACIENTES,GRUPOS_EDAD_ENCODED,GRUPOS_DIAGNOSTICOS_CIE_ENCODED
0,202301,HOSPITAL NACIONAL ARZOBISPO LOAYZA,LIMA,LIMA,LIMA,150101,20230615,FEMENINO,DE 25 A 29 AÑOS,INFECCIÓN POR SALMONELLA,OTRAS INFECCIONES DEBIDAS A SALMONELLA,1,6.0,0.0
1,202301,HOSPITAL NACIONAL ARZOBISPO LOAYZA,LIMA,LIMA,LIMA,150101,20230615,FEMENINO,MENORES DE 1 AÑO,OTRAS INFECCIONES INTESTINALES BACTERIANAS ESP...,OTRAS INFECCIONES INTESTINALES BACTERIANAS,1,0.0,1.0
2,202301,HOSPITAL NACIONAL ARZOBISPO LOAYZA,LIMA,LIMA,LIMA,150101,20230615,MASCULINO,DE 60 A 64 AÑOS,ABSCESO AMEBIANO DEL PULMÓN,AMEBIASIS,2,13.0,2.0
3,202301,HOSPITAL NACIONAL ARZOBISPO LOAYZA,LIMA,LIMA,LIMA,150101,20230615,MASCULINO,DE 65 AÑOS A MÁS,"AMEBIASIS, NO ESPECIFICADA",AMEBIASIS,1,14.0,3.0
4,202301,HOSPITAL NACIONAL ARZOBISPO LOAYZA,LIMA,LIMA,LIMA,150101,20230615,FEMENINO,DE 5 A 9 AÑOS,GIARDIASIS [LAMBLIASIS],OTRAS ENFERMEDADES INTESTINALES DEBIDAS A PROT...,2,2.0,4.0


## Encoding del target "CATEGORIA_DIAGNOSTICO"

In [141]:
categorias_diag_tar = []
for elemento in df_infecciones['CATEGORIA_DIAGNOSTICO']:
    if elemento not in categorias_diag_tar:
        categorias_diag_tar.append(elemento)


label_encoder = OrdinalEncoder(categories=[categorias_diag])
df_infecciones_clean = df_infecciones
df_infecciones_clean['GRUPOS_DIAGNOSTICOS_ENCODED'] = label_encoder.fit_transform(df_infecciones_clean[['DIAGNOSTICOS_CIE']])
df_infecciones_clean.head()

,PERIODO_DIAGNOSTICO,ESTABLECIMIENTO,DEPARTAMENTO,PROVINCIA,DISTRITO,UBIGEO,FECHA_CORTE,SEXO,GRUPOS_POR_EDAD,DIAGNOSTICOS_CIE,CATEGORIA_DIAGNOSTICO,TOTAL_PACIENTES,GRUPOS_EDAD_ENCODED,GRUPOS_DIAGNOSTICOS_CIE_ENCODED,GRUPOS_DIAGNOSTICOS_ENCODED
0,202301,HOSPITAL NACIONAL ARZOBISPO LOAYZA,LIMA,LIMA,LIMA,150101,20230615,FEMENINO,DE 25 A 29 AÑOS,INFECCIÓN POR SALMONELLA,OTRAS INFECCIONES DEBIDAS A SALMONELLA,1,6.0,0.0,0.0
1,202301,HOSPITAL NACIONAL ARZOBISPO LOAYZA,LIMA,LIMA,LIMA,150101,20230615,FEMENINO,MENORES DE 1 AÑO,OTRAS INFECCIONES INTESTINALES BACTERIANAS ESP...,OTRAS INFECCIONES INTESTINALES BACTERIANAS,1,0.0,1.0,1.0
2,202301,HOSPITAL NACIONAL ARZOBISPO LOAYZA,LIMA,LIMA,LIMA,150101,20230615,MASCULINO,DE 60 A 64 AÑOS,ABSCESO AMEBIANO DEL PULMÓN,AMEBIASIS,2,13.0,2.0,2.0
3,202301,HOSPITAL NACIONAL ARZOBISPO LOAYZA,LIMA,LIMA,LIMA,150101,20230615,MASCULINO,DE 65 AÑOS A MÁS,"AMEBIASIS, NO ESPECIFICADA",AMEBIASIS,1,14.0,3.0,3.0
4,202301,HOSPITAL NACIONAL ARZOBISPO LOAYZA,LIMA,LIMA,LIMA,150101,20230615,FEMENINO,DE 5 A 9 AÑOS,GIARDIASIS [LAMBLIASIS],OTRAS ENFERMEDADES INTESTINALES DEBIDAS A PROT...,2,2.0,4.0,4.0


In [142]:
x = df_infecciones_clean['GRUPOS_DIAGNOSTICOS_CIE_ENCODED']
y = df_infecciones_clean['GRUPOS_DIAGNOSTICOS_ENCODED']

# Modelo de clasificacion KNN

El modelo KNN, o "k-nearest neighbors", es una elección sólida en muchas aplicaciones de aprendizaje automático debido a su simplicidad y eficacia. Este algoritmo se utiliza para clasificar datos en función de su similitud con ejemplos de entrenamiento previamente etiquetados. Su justificación radica en su capacidad para adaptarse a una amplia variedad de problemas, ya que no asume ninguna distribución particular de los datos y puede manejar tanto datos numéricos como categóricos. Además, KNN es especialmente útil cuando se trata de conjuntos de datos pequeños o ruidosos, ya que puede capturar patrones locales en lugar de depender de una representación global de los datos. En resumen, KNN es una herramienta valiosa en el arsenal de algoritmos de aprendizaje automático para tareas de clasificación y regresión, y su elección se justifica por su versatilidad y facilidad de implementación.

In [143]:
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.decomposition import PCA
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
from sklearn.naive_bayes import GaussianNB

x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.2,random_state=1)

# Antes de aplicar el escalado, reformatea x_train y x_test
x_train = x_train.values.reshape(-1, 1)
x_test = x_test.values.reshape(-1, 1)

scaler=StandardScaler()

x_train=scaler.fit_transform(x_train)
x_test=scaler.transform(x_test)

model=KNeighborsClassifier()
model.fit(x_train,y_train)

y_pred=model.predict(x_test)

## Verificación de la calidad del modelo

### Estadísticas del modelo

In [144]:
from sklearn.metrics import classification_report
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

         4.0       1.00      1.00      1.00         2
         7.0       1.00      1.00      1.00         1
         8.0       1.00      1.00      1.00         1
        11.0       1.00      1.00      1.00         1
        14.0       1.00      1.00      1.00         2
        18.0       0.00      0.00      0.00         0
        20.0       0.00      0.00      0.00         1
        23.0       0.00      0.00      0.00         1
        25.0       0.00      0.00      0.00         0
        26.0       0.00      0.00      0.00         1
        28.0       1.00      1.00      1.00         1
        30.0       0.00      0.00      0.00         1
        31.0       0.25      1.00      0.40         1
        33.0       0.00      0.00      0.00         1
        34.0       0.00      0.00      0.00         1
        38.0       1.00      1.00      1.00         5
        40.0       0.00      0.00      0.00         0
        42.0       0.00    

c:\Users\alons\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\alons\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\alons\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifie

# Opinión sobre los features brindados 

Luego de completar el Challenge N°2, podemos llegar a la conclusión de que el conjunto de datos utilizado no resultó ser el más apropiado para nuestra tarea de análisis. Esto se debe principalmente a que la mayoría de las características eran cualitativas y no presentaban una correlación evidente con el objetivo que habíamos seleccionado. En consecuencia, fue necesario recurrir a la intuición y al análisis exhaustivo para identificar posibles relaciones y construir un modelo adecuado. La escasa disponibilidad de datos cuantitativos limitó nuestra capacidad para aplicar métodos estadísticos tradicionales y nos llevó a depender en gran medida de suposiciones y juicios subjetivos en la construcción del modelo.

El hecho de que nuestro modelo haya alcanzado solo un 68% de precisión en las predicciones también respalda la idea de la dificultad que implicaba encontrar una relación clara entre las características disponibles y el objetivo deseado. Esta baja precisión sugiere que nuestro modelo no pudo capturar de manera efectiva las complejas interacciones subyacentes en los datos. A pesar de estos desafíos, el proceso de trabajo con este conjunto de datos nos brindó una valiosa experiencia en la exploración de datos, la ingeniería de características y la adaptación de modelos a situaciones con datos cualitativos y escasos.